# ***Install Libs***

In [1]:
%%capture
#dry_run = False
!pip install ../input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install ../input/kornia-loftr/kornia_moons-0.1.9-py3-none-any.whl

# for depth estimation module
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp -r ../input/midasdepthestimation/MiDaS-master  /root/.cache/torch/hub/intel-isl_MiDaS_master
!cp ../input/midasdepthestimation/dpt_large-midas-2f21e586.pt  /root/.cache/torch/hub/checkpoints/
!pip install ../input/midasdepthestimation/timm-0.5.4-py3-none-any.whl

# ***Import dependencies***

In [2]:
import os
import sys
import csv
from glob import glob
import gc
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np
import cv2
import torch

# for LoFTR
import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF

# for depth-estimation
import timm

In [3]:
sys.path.append("../input/imcutils")
from imc_metric import EvaluateSubmission, ReadCovisibilityData, FlattenMatrix, LoadCalibration

sys.path.append("../input/super-glue-pretrained-network")
from models.matching import Matching as SuperGlue
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

In [4]:
# for pytorch3d
sys.path.append("../input/pytorch3ddependencies/pytorch3d_dependencies")
os.environ["CUB_HOME"] = "../input/pytorch3ddependencies/pytorch3d_dependencies/cub-1.10.0"
import pytorch3d

In [5]:
from pytorch3d.renderer.cameras import (
    PerspectiveCameras,
)
from pytorch3d.transforms.so3 import (
    so3_exp_map
)

# ***Model***

In [6]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# LoFTR
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("../input/kornia-loftr/loftr_outdoor.ckpt")['state_dict'])
matcher = matcher.to(device)
matcher.eval()
print()

In [7]:
# Super Glue
config = {
    "superpoint": {
        "nms_radius": 4,
        "keypoint_threshold": 0.005,
        "max_keypoints": 1024
    },
    "superglue": {
        "weights": "outdoor",
        "sinkhorn_iterations": 20,
        "match_threshold": 0.2,
    }
}
superglue = SuperGlue(config).eval().to(device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [8]:
def match(img_path0, img_path1, matcher, device=device):
    img0 = load_torch_image(img_path0)
    img1 = load_torch_image(img_path1)
        
    input_dict = {"image0": K.color.rgb_to_grayscale(img0).to(device), 
                  "image1": K.color.rgb_to_grayscale(img1).to(device)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
    confidence = correspondences['confidence'].cpu().numpy()
        
    return mkpts0, mkpts1, confidence

def superglue_match(img_path0, img_path1, matcher, device=device):
    resize = [-1, ]
    resize_float = True
    image_1, inp_1, scales_1 = read_image(img_path0, device, resize, 0, resize_float)
    image_2, inp_2, scales_2 = read_image(img_path1, device, resize, 0, resize_float)
    
    pred = matcher({"image0": inp_1, "image1": inp_2})
    pred = {k: v[0].detach().cpu().numpy() for k, v in pred.items()}
    kpts1, kpts2 = pred["keypoints0"], pred["keypoints1"]
    matches, conf = pred["matches0"], pred["matching_scores0"]
    
    valid = matches > -1
    mkpts1 = kpts1[valid]
    mkpts2 = kpts2[matches[valid]]
    
    return mkpts1, mkpts2

def get_F_matrix(mkpts0, mkpts1):
    # Make sure we do not trigger an exception here.
    if len(mkpts0) > 8:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.200, 0.9999, 250000)
        assert F.shape == (3, 3), 'Malformed F?'
    else:
        F = np.zeros((3, 3))
    return F

In [9]:
model_type = "DPT_Large" 
depth_estimator = torch.hub.load("intel-isl/MiDaS", model_type)
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
depth_estimator.to(device)
depth_estimator.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

def estimate_depth(filepath, depth_estimator, transform):
#     filepath = f'{src}/test_images/{batch_id}/{image_1_id}.png'
    img = cv2.imread(filepath)
    scale = 640 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    resized_img = cv2.resize(img, (w, h))
    input_batch = transform(resized_img).to(device)
    
    with torch.no_grad():
        mask = depth_estimator(input_batch)

        mask = torch.nn.functional.interpolate(
            mask.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    return mask.cpu().numpy()

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


## ***Utils***

In [10]:
src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]


def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])


def load_torch_image(fname):
    img = cv2.imread(fname)
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

In [11]:
def get_intrinsics(k):
    return np.array(k).reshape(3, 3)

def get_extrinsics(r, t):
    R = np.array(r).reshape(3, 3)
    T = np.array(t)
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = R
    extrinsic[:3, 3] = T
    return extrinsic

def skew(x):
    return np.array([[0, -x[2], x[1]],
                     [x[2], 0, -x[0]],
                     [-x[1], x[0], 0]])

def make_K(cam):
    return np.array([
        [cam.focal_length.detach().numpy()[0, 0], 0, cam.principal_point.detach().numpy()[0, 0]],
        [0, cam.focal_length.detach().numpy()[0, 1], cam.principal_point.detach().numpy()[0, 1]],
        [0, 0, 1]
    ])
    
def calc_F(cam1, cam2):
    
    intrinsic1 = get_intrinsics(make_K(cam1))
    intrinsic2 = get_intrinsics(make_K(cam2))
    extrinsic1 = get_extrinsics(cam1.R.detach().numpy(), cam1.T.detach().numpy())
    extrinsic2 = get_extrinsics(cam2.R.detach().numpy(), cam2.T.detach().numpy())
    relative = extrinsic2.dot(np.linalg.inv(extrinsic1))

    R = relative[:3, :3]
    T = relative[:3, 3]
    tx = skew(T)
    E = np.dot(tx, R)
    F = np.linalg.inv(intrinsic2).T.dot(E).dot(np.linalg.inv(intrinsic1))
    return F
    
# intrinsic1 = get_intrinsics(k1)
# intrinsic2 = get_intrinsics(k2)

# extrinsic1 = get_extrinsics(r1,t1)
# extrinsic2 = get_extrinsics(r2,t2)

# relative = extrinsic2.dot(np.linalg.inv(extrinsic1))

# R = relative[:3, :3]
# T = relative[:3, 3]
# tx = skew(T)
# E = np.dot(tx, R)
# F = np.linalg.inv(intrinsic2).T.dot(E).dot(np.linalg.inv(intrinsic1))

In [12]:
def init_values():
    N=2
#     log_R_absolute_init = torch.randn(N, 3, dtype=torch.float32, device=device)
    R_I = torch.eye(3,3).unsqueeze(0)
    R_absolute_init = torch.cat((R_I, R_I), 0)
    T_absolute_init = torch.randn(N, 3, dtype=torch.float32, device=device)
    T_absolute_init[0, :] = 0.
    focal_length_init = torch.ones((N, 2), dtype=torch.float32, device=device)

    R_absolute = R_absolute_init.clone().detach()
    R_absolute.requires_grad = True
    T_absolute = T_absolute_init.clone().detach()
    T_absolute.requires_grad = True
    focal_length = focal_length_init.clone().detach()
    focal_length.requires_grad = True
    
    return R_absolute, T_absolute, focal_length

def loss_function(xyz_unproj_world, loss_fn=torch.nn.L1Loss(reduction='mean')):
    loss = loss_fn(xyz_unproj_world[0], xyz_unproj_world[1])
    return loss
    
def optimization(R_absolute, T_absolute, focal_length, principal_points, image_sizes, xy_depth, n_iter = 2000):
    
    optimizer = torch.optim.SGD([R_absolute, T_absolute, focal_length], lr=1, momentum=0.9)
    
    camera_mask = torch.ones(2, 1, dtype=torch.float32, device=device)
    
    for it in range(n_iter):
        cameras_absolute = PerspectiveCameras(
            R = R_absolute,
            T = T_absolute,
            focal_length = focal_length,
            principal_point = principal_points,
            image_size = image_sizes,
            in_ndc = False,
            device = device,
        )
        optim_one_iter(cameras_absolute, xy_depth, optimizer)
        
    return cameras_absolute
    
    
def optim_one_iter(cameras, xy_depth, optimizer):
    optimizer.zero_grad()
    xyz_unproj_world = cameras.unproject_points(xy_depth, world_coordinates=True)
    loss = loss_function(xyz_unproj_world, loss_fn=torch.nn.MSELoss(reduction='mean'))
    loss.backward()
    optimizer.step()

In [13]:
def make_xy_depth(dic, device=device):
    depth1 = dic["depth1"]
    pts1 = dic["points1"].astype(int)
    pts1_depth = torch.Tensor(depth1[pts1[:,1], pts1[:,0]])
    pts1 = torch.Tensor(pts1)
    
    depth2 = dic["depth2"]
    pts2 = dic["points2"].astype(int)
    pts2_depth = torch.Tensor(depth2[pts2[:,1], pts2[:,0]])
    pts2 = torch.Tensor(pts2)
    
    # discard points where depth == 0
    valid_idxs = (pts1_depth > 0) & (pts2_depth > 0)
    pts1_depth = pts1_depth[valid_idxs]
    pts2_depth = pts2_depth[valid_idxs]
    pts1_x = pts1[:, 0][valid_idxs]
    pts1_y = pts1[:, 1][valid_idxs]
    pts1 = torch.cat((pts1_x.unsqueeze(1), pts1_y.unsqueeze(1)), dim=1)
    pts2_x = pts2[:, 0][valid_idxs]
    pts2_y = pts2[:, 1][valid_idxs]
    pts2 = torch.cat((pts2_x.unsqueeze(1), pts2_y.unsqueeze(1)), dim=1)
    
    xy_depth1 = torch.cat((pts1, pts1_depth.unsqueeze(1)), dim=1).unsqueeze(0)
    xy_depth2 = torch.cat((pts2, pts2_depth.unsqueeze(1)), dim=1).unsqueeze(0)
    xy_depth = torch.cat((xy_depth1, xy_depth2), dim=0).to(device)
    xy_depth.requires_grad = False
    return xy_depth

def compute_F_from_depth_points_dic(depth_points_dic, n_iter=500):
    xy_depth = make_xy_depth(depth_points_dic)
    
    principal_point1 = depth_points_dic["principal_point1"].unsqueeze(0)
    principal_point2 = depth_points_dic["principal_point2"].unsqueeze(0)
    principal_points = torch.cat((principal_point1, principal_point2), dim=0).to(device)
    principal_points.requires_grad = False
    
    image_size1 = depth_points_dic["image_size1"].unsqueeze(0)
    image_size2 = depth_points_dic["image_size2"].unsqueeze(0)
    image_sizes = torch.cat((image_size1, image_size2), dim=0).to(device)
    image_sizes.requires_grad = False
    
    log_R_absolute, T_absolute, focal_length = init_values()
    
    optimized_cameras = optimization(log_R_absolute, T_absolute, focal_length, principal_points, image_sizes, xy_depth, n_iter=n_iter)
    print()
    return calc_F(optimized_cameras[0].cpu(), optimized_cameras[1].cpu())

# ***Inference***

In [14]:
F_dict = {}

depth_points_dict = {}

import time
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row
    # Load the images.
    st = time.time()
    image1_filepath = f'{src}/test_images/{batch_id}/{image_1_id}.png'
    image2_filepath = f'{src}/test_images/{batch_id}/{image_2_id}.png'
    image_1 = load_torch_image(image1_filepath).to(device)
    image_2 = load_torch_image(image2_filepath).to(device)
    
    depth1 = estimate_depth(image1_filepath, depth_estimator, transform)
    depth2 = estimate_depth(image2_filepath, depth_estimator, transform)
    
    #LoFTR
    mkpts0, mkpts1, confidence = match(image1_filepath, image2_filepath, matcher, device)
    max_n_points = 50
    sorted_idxs = np.argsort(confidence)[::-1]
    idx_to_use = sorted_idxs[:min(max_n_points, len(sorted_idxs))]
    mkpts0 = mkpts0[idx_to_use]
    mkpts1 = mkpts1[idx_to_use]
    
    
#     #SuperGlue
#     sg_mkpts0, sg_mkpts1 = superglue_match(f'{src}/test_images/{batch_id}/{image_1_id}.png', f'{src}/test_images/{batch_id}/{image_2_id}.png', superglue, device)
    
#     mkpts0 = np.vstack((loftr_mkpts0, sg_mkpts0))
#     mkpts1 = np.vstack((loftr_mkpts1, sg_mkpts1))

    img1_h, img1_w = image_1.shape[2], image_1.shape[3]
    img2_h, img2_w = image_2.shape[2], image_2.shape[3]

    depth_points_dic = {
        "depth1": depth1,
        "depth2": depth2,
        "image1_filepath": image1_filepath,
        "image2_filepath": image2_filepath,
        "principal_point1": torch.Tensor((img1_w/2, img1_h/2)),
        "principal_point2": torch.Tensor((img2_w/2, img2_h/2)),
        "image_size1": torch.Tensor((img1_h, img1_w)),
        "image_size2": torch.Tensor((img2_h, img2_w)),
        "points1": mkpts0,
        "points2": mkpts1,
    }
    try:
        F_dict[sample_id] = compute_F_from_depth_points_dic(depth_points_dic, n_iter=500)
    except:
        F_dict[sample_id] = np.zeros((3, 3))
    
    gc.collect()
    
    print(f"took {time.time() - st}sec")
    
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:461.)
  return torch.floor_divide(self, other)
../input/pytorch3ddependencies/pytorch3d_dependencies/pytorch3d/transforms/transfor


took 9.639521598815918sec

took 2.7724316120147705sec

took 2.4895660877227783sec


In [15]:
n_max_points = 50
confidence.shape
sorted_idxs = np.argsort(confidence)[::-1]
idx_to_use = sorted_idxs[:min(n_max_points, len(sorted_idxs))]
idx_to_use

array([ 59, 171,  71,  55,  57,  45,  91,  68,  79,  78,  93,  54,  56,
        60, 136,  70, 145,  90, 123,  31, 139, 116, 193, 202, 182, 151,
        13, 102, 237,  98,  92, 200, 100,  58,  51,  30, 104, 101, 105,
        53, 159,  17,  99,  50,  82,  89,  12,   5, 115, 188])

In [16]:
# mkpts0.shape
# confidence[idx_to_use]
# mkpts0[idx_to_use].shape
# a = np.ones(10)[:10]
# a.shape

In [17]:
# print(optimized_cameras[0].R)
# print(optimized_cameras[0].T)
# print(optimized_cameras[0].K)
# print(optimized_cameras[0].focal_length)
# print(optimized_cameras[0].principal_point)
# print(optimized_cameras[0].image_size)
# print()
# print(optimized_cameras[1].R)
# print(optimized_cameras[1].T)
# print(optimized_cameras[1].K)
# print(optimized_cameras[1].focal_length)
# print(optimized_cameras[1].principal_point)
# print(optimized_cameras[1].image_size)

In [18]:
# dir(optimized_cameras[0])